In [1]:
import pandas as pd
from normalization_PSMSHandler import *

from mainSearchFunctions import *
from postSearchProcessing import *

import re, numpy as np
import math
import os
import os.path
import matplotlib.pyplot as plt
import sys
import time
from logFunctions import *
pd.options.mode.chained_assignment = None  # default='warn'

import configparser
config = configparser.ConfigParser()
from os.path import dirname

In [2]:
start_time = time.time()

In [3]:
os.getcwd()

'/Users/spoudel1/Desktop/JUMPp_lib_program/JumplibrarySearchParallel_TwoScores'

In [4]:
# # Read the parameter file using configparser
params_file = "../Test_search_parallel_8cores/specLib_firstSearch.params"

In [5]:

config.read(params_file)

# exp_mzxml = config["specLib"]["exp_mzxml"] #raw file
exp_ms2 = config["specLib"]["exp_ms2"] #raw file
specLibFolder = config["specLib"]["specLibFolder"] #library containing folder
ms1_tol = float(config["specLib"]["ms1_tol"]) #precursor ion tolerance

top_ions_control = config["specLib"]["top_ions_control"]
top_ions_min_max = config["specLib"]["top_ions"] #total number of top library ions that should be present in spectrum 
ms2_tol = float(config["specLib"]["ms2_tol"]) #fragment ion tolerance
# rt_fdr = float(config["specLib"]["rt_fdr"]) #rt tolerance in SD
#window number determines the lenght of mz for normalization of intensity
window_number = int(config["specLib"]["window_number"])

binsize = config["specLib"]["binsize"] #binning width for selecting top ions in the ms2 spectrum 
top_ions_per_bin = int(config["specLib"]["top_ions_per_bin"]) #maximum fragment ions that are selected from raw spectrum for each bin

start_end_scan_range = config["specLib"]["start_end_scan_range"] #start and end scans supplied by users

# outputFolder = fileroot+"/"+config["specLib"]["outputFolder"] #search results are stored in this folder

#score method
method = config["specLib"]["method"]
null_search = config["specLib"]["null_search"]
sim_mass = float(config["specLib"]["sim_mass"])
n_cores = int(config["specLib"]["n_cores"])

In [6]:
null_search ="0"
n_ions = 20
topRanks = 10

In [7]:
#THis is additional step for local work as we use the program from jupyter notebook
os.chdir(dirname(exp_ms2))

In [8]:

outputFolder = exp_ms2.split("/")[-1].split(".ms2")[0]
makedirectory(outputFolder)


In [9]:
#check null search and if null_search = 1, sim_mass needs to be added else it is 0
if null_search == "0":
    sim_mass = 0.0

#tolerace type for dynamic or static tolerance

tolerance_type = config["specLib"]["tolerance_type"]

#Dynamice tolerance file if dynamic tolerance is selected

dyn_tol_file = config["specLib"]["dyn_tol_file"]


#tolerance for ms2 matching = fragment ion tolerance
tol = float(config["specLib"]["tol"])
#example pepxml file to parse the modification information
tmt = config["specLib"]["tmt"]

#Let us first look at the all protein ppml file

libtype = float(config["specLib"]["libtype"])

min_top_ions = int(top_ions_min_max.split(",")[0])
top_ions = int(top_ions_min_max.split(",")[1])


#this dictionaryneeds to be updated with other datatype
libtypeDict = {1:"tmt18_default", 2:"tmt11_default", 3:"tmt18_pho", 4:"tmt11_pho", 5:"tmt18_ub", 6:"tmt11_ub", 100:"labelfree_default", 1000:"silaclys_default"}
libtypename = libtypeDict[int(libtype)]

In [10]:
#take all protein ppml file first
all_prot_ppml = specLibFolder+"/intermediate/id_all_pep.ppml"
#make all protein ppml file dataframe
all_prot_ppmlDF = fileToDF(all_prot_ppml)
id_prot = all_prot_ppmlDF[["PeptideSeqWithRealDelMass","Protein Accession #"]]
#rename names for consistency and groupby peptide ID and get accession separated by ,
idProtDF = id_prot.groupby('PeptideSeqWithRealDelMass').agg( lambda x: ','.join(list(x))).reset_index()
#protein maped to peptide dataframe
# print (idProtDF.columns)
idProtDF.rename(columns = {"PeptideSeqWithRealDelMass":"Peptide","Protein Accession #":"Protein"}, inplace=True)
L_ID_allProtDict = dict(zip(idProtDF.Peptide, idProtDF.Protein))


In [11]:
#set current directory as working directory
# path = os.getcwd()
# os.chdir(path)

# exp_ms2 = "/Users/spoudel1/Desktop/JUMP_specLib/Program/ms2/preprocess/FTLD_Batch2_F50.ms2"
specLib = specLibFolder+"/jumplib_human_{}.splib".format(libtypename)
# specLib = "/Users/spoudel1/Desktop/JUMP_specLib/specLib/OneFraction/SpectralLibraryTargetDecoy.spLib"


#expMZXML = exp_ms2.split("/")[-1].split(".")[0]


In [12]:
int_sd_dict = {}
if tolerance_type.upper() == "DYNAMIC":
    int_sd_dict = parseDynamicIntensityFile(dyn_tol_file,tol)


logFile = outputFolder+"/jump_lib_s.log"

#removing log file if previously present
try:
    rmFile(logFile)
except:
    print ("No jump_lib_s.log is present. Writing log file now")



# filename = exp_ms2.split("/")[-1]


In [13]:
write_log (logFile,"Parsing experimental ms2 file and storing as dataframe. Below is the summary")
write_log (logFile,"Name of input file = {}".format(exp_ms2))
write_log (logFile,"This will take time depending on size of ms2 file")

#simulated mass changes the precursor m/z by sim_mass dalton. This simulation is just done at the input file not in the library
expDF_all = ms2ToDf_spec(exp_ms2, sim_mass)
write_log (logFile,"\nThe total number of precursor candidates (dtas) are {}".format(expDF_all.shape[0]))

write_log (logFile,"\nParsing Spectral Library database and storing as dataframe. Below is the summary")
write_log (logFile,"This will take time depending on size of ms2 file")


Parsing experimental ms2 file and storing as dataframe. Below is the summary
Name of input file = /Users/spoudel1/Desktop/Spectral_Library_Search/htmt_301/htmt_b301_f001.ms2
This will take time depending on size of ms2 file

The total number of precursor candidates (dtas) are 32358

Parsing Spectral Library database and storing as dataframe. Below is the summary
This will take time depending on size of ms2 file


In [14]:
#check to see if the user has supplied start and end scans

start_end = start_end_scan_range.split(",")
start_scan = int(start_end[0])
end_scan = int(start_end[1])

#parse the expDF_all to get the start and end scans only

if end_scan != 0:
    expDF = expDF_all.loc[(expDF_all.scan >= start_scan) & (expDF_all.scan <= end_scan)]
else:
    expDF = expDF_all

In [15]:
#There should be a pickle file in the specLibFolder but just to make sure
if os.path.exists(specLibFolder+"/jumplib_human_{}.pkl".format(libtypename)):
    libDF = pd.read_pickle(specLibFolder+"/jumplib_human_{}.pkl".format(libtypename))
else:
    libDF = ms2ToDf_spec(specLib)
   #outfilename = specLibFolder+"/jumplib_human_{}.splib".format(libtypename) 
    libDF.to_pickle(specLibFolder+"/jumplib_human_{}.pkl".format(libtypename))


In [16]:
write_log (logFile,"\nThe total TARGET entries in Library are {}".format(libDF.shape[0]/2))



The total TARGET entries in Library are 383526.0


In [17]:
if method != "normalized_dot_product":
    write_log (logFile,"Since the scoring method is not normalized dot product. We are normalizing the intensity. Performing normalization step\n")
    normalizeIntensity(expDF,window_number)
    normalizeIntensity(libDF,window_number)


In [18]:
if tolerance_type.upper() == "DYNAMIC":
    logTransformMS2Intensity(expDF)#this is for dynamic intensity tolerance


In [19]:
write_log (logFile,"\nLibrary searching is underway. Below are the list of major steps performed")
write_log (logFile,"Matching Precursor ions with ms1 tolerace (ppm) = {}".format(ms1_tol))
write_log (logFile,"Matching product ions")
write_log (logFile,"Searching for top matching fragment ions {}".format(top_ions))
write_log (logFile,"If found, cleaning the library entries for redundancy")
write_log (logFile,"Preprocessing input spectra for pattern matching")
write_log (logFile,"For each library product ions, look for ions with tolerance (ppm) = {}".format(ms2_tol))
write_log (logFile,"If no match found intensity = 0 to that product ion\n\n")



Library searching is underway. Below are the list of major steps performed
Matching Precursor ions with ms1 tolerace (ppm) = 10.0
Matching product ions
Searching for top matching fragment ions 3
If found, cleaning the library entries for redundancy
Preprocessing input spectra for pattern matching
For each library product ions, look for ions with tolerance (ppm) = 10.0
If no match found intensity = 0 to that product ion




In [20]:
def select_TopN_Lib_ions(exp_mz_list, intensity_exp, matched_library_DF, n_ions, ms2_tol, topRanks=10): #top_ion is the minimum no of ions (intensity ranked top) that should be present in experimental mz 
    
    
    dotProductList = [] #this checks if top_ion are present in exp_mz_list
    
    #this is to speed up the scanning process as numpy array are fastest
    mz_cols = list(matched_library_DF.columns) #for indexing columns
    np_arr = matched_library_DF.to_numpy() #numpy array conversion of dataframe
    for row in np_arr: #each matched library checked for top ions according to parameters above
        mz = row[mz_cols.index("m/z")] #mz list (ms2)
        intensity = row[mz_cols.index("intensity")] #intensity list (ms2)
        scan = str(row[mz_cols.index("scan")]) #scan 
       
        ind = np.argsort([-i for i in intensity]) #sorting by descending order intensity
        

        top_ion_lib = {"mz":[],"intensity":[]} #empty dict for top ion collection
        
        top_ion_lib["mz"] = [mz[i] for i in ind[0:n_ions]]
        top_ion_lib["intensity"] = [intensity[i] for i in ind[0:n_ions]]
        
        
        cnt = 0
        #make a trimeed spectral dictionary and update it as the ion is within the tolerance
        tr_featSpec = {}


        
        for ion in top_ion_lib["mz"]: #looping over top_ions to see if they are present in experimental mz
            for index, masses in enumerate(exp_mz_list):
                massshift = ppmCalc(float(ion), float(masses), ms2_tol)
                if abs(massshift) < ms2_tol:
                    if ion not in tr_featSpec.keys():
                        tr_featSpec[ion]=intensity_exp[index]
                    
                        
                    else:
                        old_intensity = tr_featSpec[ion]
                        
                        if intensity_exp[index] > old_intensity:
                            tr_featSpec[ion] = intensity_exp[index] 
        
        
#         print ("Top library ions ",top_ion_lib["mz"])
        
        tr_featSpec2 = {"mz":[],"intensity":[]}
        
        for ions in top_ion_lib["mz"]:
            #add intensity 0 to all the ions that are present in library but are not within the given tolerance in spectrum raw
            if ions in tr_featSpec.keys():
                tr_featSpec2["mz"].append(ions)
                tr_featSpec2["intensity"].append(tr_featSpec[ions])
            else:
                tr_featSpec2["mz"].append(ions)
                tr_featSpec2["intensity"].append(0.0)

#         print (tr_featSpec2)
        
        #compute normalized dot product here
        spec_query = np.array(tr_featSpec2["intensity"])
        spec_reference = np.array(top_ion_lib["intensity"])
        
        dp = normalizedDotProduct(spec_query,spec_reference)
        dotProductList.append(dp)
        
    matched_library_DF["QuickDotProduct"] = dotProductList
    #Select top 10 Ranks
    matched_lib_DF_top = matched_library_DF.sort_values(by=["QuickDotProduct"], ascending=False)
        
    return matched_lib_DF_top.iloc[0:topRanks]


In [53]:
expDF = expDF_all.iloc[0:10000]

In [46]:
def search(expDF,libDF,n_ions, ms2_tol,ms1_tol, int_sd_dict,topRanks,method="normalized_dot_product"):
    
    mz_cols = list(expDF.columns)
    np_arr = expDF.to_numpy()

    dot_product_results = {}
    cnt = 0
    update = 10000
    
    for row in np_arr:
        scan = str(row[mz_cols.index("scan")])
        charge = int(row[mz_cols.index("charge")])
        precMZ = float(row[mz_cols.index("prec_MZ")])
        mz = row[mz_cols.index("m/z")]
        
        intensity = list(row[mz_cols.index("intensity")])
        matched_lib_DF = scanPrecursorMatch(precMZ, charge, libDF, ms1_tol)
        
        
        if matched_lib_DF.shape[0] >=1: #there may not be any matches to the library so we have to check the library match to avoid error
            matched_lib_DF_top2 = select_TopN_Lib_ions(mz, intensity, matched_lib_DF, n_ions, ms2_tol, topRanks)
            spectrumInputDict = {"mz":mz,"intensity":intensity}
            tolInputDict = {}
            cnt+=1
            
            mz_cols2 = list(matched_lib_DF_top2.columns)
            np_arr2 = matched_lib_DF_top2.to_numpy()
            for row2 in np_arr2: 
                #removal of library cleaning step enables us to directly generate lib_mz_int_dict
                #lib_mz_int_dict2 = {}
                lib_mz_int_dict = {}
                L_ID = row2[mz_cols2.index("L_ID")]
                RT = row2[mz_cols2.index("RT")]
                L_peptide = row2[mz_cols2.index("L_peptide")]
                prec_mz = row2[mz_cols2.index("prec_MZ")]
    #             print(L_peptide)

                #library cleaning step is removed so lib_mz_int_dict2 can be saved as lib_mz_int_dict

                lib_mz_int_dict["mz"] = row2[mz_cols2.index("m/z")]

                # print ("The library entry is {} and peptide is {}".format(L_ID, L_peptide))
                # print ("Length of library product ions = {}".format(len(lib_mz_int_dict2["mz"])))

                if method == "normalized_dot_product":
                    lib_mz_int_dict["intensity"] = list(row2[mz_cols2.index("intensity")]) #use normalize dictionary for the library input
                else:
                    lib_mz_int_dict["intensity"] = list(row2[mz_cols2.index("normalized_intensity")]) #use normalize dictionary for the library input            
    #             #tolerance_type defines how to choose the tolerance Dynamic will use the int_sd_dict dictionary else static tolerace will be used
                #clean library ions too to check for ions within mass tolerance           
                

                #may be this is not required as we have all theoretical ions
                #lib_mz_int_dict = cleanLibRedundancy(lib_mz_int_dict2, ms2_tol)
                

                tr_featSpec = trimFeatSpec(spectrumInputDict,tolInputDict,int_sd_dict,tolerance_type,lib_mz_int_dict,ms2_tol) #int_sd_dict = tolerance SD defined intensity from the file after mass calibration
                #dp = calcMS2Similarity(tr_featSpec,lib_mz_int_dict,ms2_tol)
                #input for other scoring techniques
                spec_query = np.array(tr_featSpec["intensity"])
                spec_reference = np.array(lib_mz_int_dict["intensity"])
                #print (spec_query)
                #print (spec_reference)

                #if method == "JiHoon_Norm_DP":
                #    dp = calcMS2Similarity(tr_featSpec,lib_mz_int_dict,ms2_tol)


                if method == "normalized_dot_product":
                    dp = normalizedDotProduct(spec_query,spec_reference)

                else:
                    spec_query_ = conversionDictSpecToNumpyArrayFormat(tr_featSpec)
                    spec_reference_ = conversionDictSpecToNumpyArrayFormat(lib_mz_int_dict)
		    
                    if method == "DoubleNormalizedDP":
                        dp = normalizedDotProduct(spec_query,spec_reference)
                    if method == "DP_Peng":
                        dp = DP_Peng_similarity(spec_query,spec_reference)
                    if method == "unweighted_entropy":
                        dp = unweightedEntropySimCalc(spec_query_,spec_reference_)
                    if method == "dot_product":
                        dp = dot_product_similarity(spec_query,spec_reference)
                    if method == "fidelity":
                        dp = fidelity_similarity(spec_query,spec_reference)
                    if method == "bhattacharya_2":
                        dp = bhattacharya_2_similarity(spec_query,spec_reference)

                if "Decoy_" in L_peptide:
                    L_peptide2 = L_peptide.split(";")
                    L_peptide = "{};{};{}".format(L_peptide2[0],L_peptide2[1],prec_mz)
                scanKey = str(scan)+"."+str(charge)+"."+str(precMZ)


                if scanKey not in dot_product_results.keys():
                    dot_product_results[scanKey] = [L_ID+";"+str(dp)+";"+str(RT)+";"+L_peptide]
                else:
                    dot_product_results[scanKey].append(L_ID+";"+str(dp)+";"+str(RT)+";"+L_peptide)
                     
        
        if cnt == update:
            remainingScans = len(expDF) - cnt
            print ("Total scan searched = ", cnt,"\nRemaining scans = " ,remainingScans)
            update+=10000
       
    final_result = {}
    for results in dot_product_results.keys():
    #     print (len(dot_product_results[results]))
        final_result[results] = ",".join(dot_product_results[results])
#     print (final_result) 
    return final_result


In [58]:
libDF

,scan,charge,[M+H]+,prec_MZ,L_ID,L_peptide,L_protein,RT,m/z,intensity
0,1,2,1103.133405,1103.133405,p0000001,A(304.2071453)AAAAAAAAAAAAAAGAGAGAK(304.207145...,sp|P55011|S12A2_HUMAN,82.245,"[348.2566209, 376.2515356, 419.2937347, 447.28...","[285448.0, 1103696.0, 96475.0, 983727.0, 13049..."
1,2,3,735.758029,735.758029,p0000002,A(304.2071453)AAAAAAAAAAAAAAGAGAGAK(304.207145...,sp|P55011|S12A2_HUMAN,82.800,"[295.1850767, 330.7036336, 348.2566209, 366.22...","[163895.0, 1547760.0, 10665.0, 4151911.0, 1537..."
2,3,2,1246.146653,1246.146653,p0000003,A(304.2071453)AAAAAAAAAAPPAPPEGASPGDSAR;2;1246...,sp|Q8WXD9|CSKI1_HUMAN,42.315,"[158.0924031, 175.1189522, 229.1295169, 246.15...","[28615.0, 191149.0, 3926.0, 30670.0, 13716.0, ..."
3,4,2,595.352256,595.352256,p0000004,A(304.2071453)AAAAAAAAAR;2;595.3522559,sp|Q9Y651|SOX21_HUMAN,18.841,"[158.0924031, 175.1189522, 229.1295169, 246.15...","[11051.0, 40249.0, 6927.0, 3775.0, 5985.0, 316..."
4,5,2,697.934198,697.934198,p0000005,A(304.2071453)AAAAAAAAK(304.2071453);2;697.934...,sp|Q99453|PHX2B_HUMAN,24.230,"[348.2566209, 376.2515356, 419.2937347, 447.28...","[13755.0, 50439.0, 4374.0, 36874.0, 39993.0, 4..."
...,...,...,...,...,...,...,...,...,...,...
767047,458010,1,1028.605961,1028.605961,Decoy_p0074484,Decoy_Y(304.2071453)VLSSR;1;1028.6059607,Decoy_sp|P12532|KCRU_HUMAN,49.753,"[158.0924031, 175.1189522, 257.160817, 274.187...","[4414.0, 163419.0, 14817.0, 46136.0, 13258.0, ..."
767048,458013,2,1634.747360,817.877318,Decoy_p0074487,Decoy_Y(304.2071453)VM(15.99492)PSC(57.02146)E...,Decoy_sp|Q9H4F8|SMOC1_HUMAN,34.761,"[158.0924031, 175.1189522, 272.1353305, 273.11...","[1242955.0, 9359339.0, 92829.0, 758519.0, 5448..."
767049,458024,2,2157.015800,1079.011538,Decoy_p0074498,Decoy_Y(304.2071453)VNM(15.99492)QDPEM(15.9949...,Decoy_sp|O00217|NDUS8_HUMAN,55.858,"[158.0924031, 175.1189522, 255.1451669, 272.17...","[15620.0, 82088.0, 28507.0, 189451.0, 32194.0,..."
767050,458025,2,2141.020880,1071.014078,Decoy_p0074499,Decoy_Y(304.2071453)VNM(15.99492)QDPEM(15.9949...,Decoy_sp|O00217|NDUS8_HUMAN,76.455,"[440.2828357, 451.3199495, 468.2777503, 539.35...","[696817.0, 375554.0, 1338830.0, 207055.0, 6126..."


In [54]:
n_ions = 10
topRanks = 5

In [55]:
results_list = []
df_split = np.array_split(expDF, 20)
cnt=1
for x in df_split:
    final_result = search(x,libDF,n_ions, ms2_tol,ms1_tol, int_sd_dict,topRanks,method="normalized_dot_product")
    print ("TOtal frames completed {}".format(cnt))
    cnt+=1
    results_list.append(final_result)

TOtal frames completed 1
TOtal frames completed 2
TOtal frames completed 3
TOtal frames completed 4
TOtal frames completed 5


KeyboardInterrupt: 

In [40]:
#5:20PM
start_time=time.time()
final_result = search(expDF,libDF,n_ions, ms2_tol,ms1_tol, int_sd_dict,topRanks,method="normalized_dot_product")
print(time.time() - start_time)

1
1
2
3
4
5
6
7
7
8
9
9
10
10
11
11
11
12
12
13
14
15
16
17
18
19
19
20
21
21
22
23
24
24
24
25
26
27
27
28
29
30
30
31
32
32
33
34
35
36
36
36
36
37
38
38
38
39
39
40
40
41
42
43
44
45
45
46
47
47
48
49
50
51
52
52
52
53
53
53
54
55
55
55
56
57
57
58
59
60
61
62
63
64
65
66
67
68
69
70
10.730446100234985


In [49]:
start_time=time.time()
#library searching results are kept as a dictionary final result
# final_result = librarySearchMain(expDF,libDF,top_ions,ms2_tol,ms1_tol,top_ions_per_bin,binsize,tolerance_type,int_sd_dict, method, sim_mass)
# final_result = librarySearchMain(expDF,libDF,top_ions,ms2_tol,ms1_tol,top_ions_per_bin,binsize,tolerance_type,int_sd_dict, method

#use multiprocessing here
#Define arguments that goes after the main dataframe that is going to be searched expDF

args = (libDF,n_ions, ms2_tol,ms1_tol, int_sd_dict,topRanks,"normalized_dot_product")

#initialize empty dictionary to update after multiple processing
final_result = {}
#calling the parrallel function. result variable will be the list of result of 4 processes
#input is the input DF for searching and remaining arguments args
result = parallelize_dataframe(search,expDF, args,  n_cores=n_cores)
for x in result:
    final_result.update(x.get()) #main work of the workers. The result is updated to the final result

    
print(time.time() - start_time,"seconds")

Process ForkPoolWorker-36:
Traceback (most recent call last):
  File "/Users/spoudel1/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()

KeyboardInterrupt

  File "/Users/spoudel1/miniconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/spoudel1/miniconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-46-db5cff2dfa94>", line 17, in search
    matched_lib_DF = scanPrecursorMatch(precMZ, charge, libDF, ms1_tol)
  File "/Users/spoudel1/Desktop/JUMPp_lib_program/JumplibrarySearchParallel_TwoScores/mainSearchFunctions.py", line 250, in scanPrecursorMatch
    libDF_matched = libDF.loc[(libDF.prec_MZ.astype("float") >= min_prec_mzCheck) & (libDF.prec_MZ.astype("float") <= max_prec_mzCheck) & (libDF.charge == charge)]
  File "/Users/spoudel1/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.p

  File "/Users/spoudel1/miniconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/Users/spoudel1/miniconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/spoudel1/miniconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/Users/spoudel1/miniconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/spoudel1/miniconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/spoudel1/miniconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/spoudel1/miniconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/spoudel1/miniconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/Users/sp

In [ ]:
write_log (logFile,"Total candidates psms (all Ranks) before filtering = {}".format(len(final_result.keys())))


In [ ]:
# consolidation of one psms to one scan for TMT data
printDF2Rank1, printDF2 = postsearchProcessing(expDF, final_result, outputFolder,tmt,logFile, exp_ms2, L_ID_allProtDict) #sta_AA helps us to see if the data is TMT or not {'K': 304.2071453, 'C': 57.02146, 'n': 304.2071453} #for tmt K and n values are equal

cmd2 = "cp "+params_file+" "+outputFolder
os.system(cmd2)


write_log (logFile,"The JUMPp-lib search is complete for {}".format(exp_ms2))
write_log (logFile,"Time taken for library search and filter --- %s seconds ---" % (time.time() - start_time))
write_log (logFile,"\n\n")
